# Using the NonLinear Prior

## Basic Non-Linear Portfolio Optimization

### Background

### High-Level Outline

When using non-linear instruments we have to be careful about how we prepare the return distributions we feed into SKfolio's optimizations. For one, we will need to entirely dispose with the usual usage-pattern of feeding a history of returns to the optimization. The structure of these non-linear instruments necessarily means that their future return distributions should differ from their historical distributions (think of a bond whose duration decreases as it approaches maturity, and therefore the volatility of its returns also decreases). Therfore, instead of using return histories, we must use return-distributions designed for a specific point in time. In the case of a bond one approach may be:

1. Transform the bond price history into z-spreads using historical discount curves.
2. Compute the daily returns of the z-spreads.
3. Multiply the current z-spreads by the historical z-spread returns to derive a distribution for tomorrow's z-spread.
4. Take the distribution of tomorrow's z-spread and convert them to bond prices, using tomorrow as the pricing date, to derive a distribution of tomorrow's bond prices.
5. Divide the distribution of tomorrow's bond prices by today's bond prices to derive a distribution of bond-returns.
6. Feed this distribution of bond returns into an optimizer.

#### Using Invariants

Note that this process relies on the key assumption that the distribution of the z-spread remains the same historically. Transforming prices of such non-linear instruments into quantities with a return distribution that, approximately, stays inCoviant historically, will be the general approach for generating the return distributions of such instruments. 

However, even if the returns of this "inCoviant" Coviable really are invariant, there remains one big bugbear that threatens to throw a wrench in the works of SKfolio's elegant design patterns: while SKfolio's optimizations library only take in returns without regard for absolute prices, the return of a derivative linked to an invariant depends on both the return of the invariant and its original magnitude. For example, a bond's duration changes with its price, meaning that its sensitivity to the return of the z-spread changes with the absolute value. Similarly, a stock option's delta changes with the absolute price of the underlying equity. So even if we have a return distribution for all the relevant invariants, we'll still need some reference prices before we're able to compute the return distribution of our actual portfolio. We will see later that with some careful manipulation of SKfolio's `Prior` interface this becomes relatively painless. 

#### What Type of Historical Return to Use?

However, this assumption may be inappropriate in many cases. Z-spreads, for example, are often thought to be mean-reverting. This means that taking 1-week or 1-month returns, for example, will yield very different price distributions than those you would get by assuming daily z-spread returns were i.i.d and extrapolating. Therefore, when we calculate the returns we will want to be able to specify the differencing period.

#### Using Different Return-Types for Different Instruments

In the above example we held rates constant. This may be reasonable if you have a low-cost, frequently-rebalanced duration hedging programme you're applying to your bond portfolio. But what if we're also interested in the interest rates risk? Then there is an important nuance to the returns on interest-rates we must consider. Because interest rates can go negative and the absolute changes in interest rates are largely independent of the current level of rates, linear returns ($\frac{\text{IR}_{t+1}}{\text{IR}_t} - 1$) are inappropriate. A better choice would be arithmetic returns: $\text{IR}_{t+1} - \text{IR}_t$. Thus we would like to be able to specify one type of return for a given set of market prices and another type of return for another set of market prices, and aggregate them all into one forward-looking return-distribution. 

#### A Note About Model Risk

Something to bear in mind is how this approach changes how we evaluate models out-of-sample. Here the approach would be to generate one return distribution using the **training** dataset and fit the optimizer on this distribution, and then to test the optimization we use the same methodology to generate a new return distribution using the **testing** dataset. Thus we are no longer evaluating the performance of the optimization on a return history either. This would be impossible because we are optimizing our portfolios for returns at a single point in time: there will only ever be one realized return at that point in time making it impossible to evaluate risk measures such as standard deviation. Using the return distribution generated using the out-of-sample dataset is, therefore, the only viable approach. However, this introduces a model risk that is not present when evaluating portfolio performance directly on the out-of-sample return history (as is the norm with equity portfolio optimizations). We are once again relying on the return history of the *invariant* vairable being random independent realizations of the same underlying probability distribution. This is never truly the case in financial time series, but it is necessary to make such an approximation. In short, be sure to understand why the returns of the invariant may not necessarily be i.i.d. and how it introduces a "model risk" into out-of-sample testing results. 

#### Handling Cashflows

Finally, there is the issue of cashflows. When we are generating a return distribution for our portfolio we must make sure not to exclude the return that comes from cash flows received during this period. What should we do with the cash we receive? One option is to set up a cash pocket and let it grow at rates. However, this would require us always feeding in overnight rate data and restructuring SKfolio's architecture a little to add cash pockets. In a perfect world we would model all cashflows precisely and reinvest them. However, this would require having the return distribution at the time of every cashflow, and setting up such a complex multi-period return distribution is likely not to be worthwhile for something that will likely only have a marginal influence on the ultimate portfolio weights. Simply adding the paid cashflows to the final prices computed in the price distribution should be a good enough approximation, even if we do miss out on some growth due to rates on the paid out coupon.

### Implementation

Notice that the above process makes important use of the notion of "today's" prices. This is not something we see in the usual equity-type usage of SKfolio. Notice also that we are using prices from instruments (interest rate swaps to build our discount curves) that are not actually in our portfolio. This is also a deviation from the usual SKfolio, where all instruments are assumed to be a part of the portfolio. These facts significantly complicate the design patterns needed to implement these transformations into the usual SKlearn / SKfolio framework.

The transformation of the prices and market context into a return distribution for the instruments in our portfolio is something we'd like to introudce as a pre-processing layer for the SKfolio optimizations that will also work as part of cross-validation. To do so, we have to design the necessary transformations such that they conform to the SKlearn `TransfomerMixin` interface. 

The `transform` method of the `TransformerMixin` takes in one dataframe and spits out one dataframe. Therefore, although we're dealing with data from many different sources, we'll keep what we can in one single large dataframe and keep what other data we need in global variables that may be updated as side effects of each transformer. Why not use metadata for passing in some of the data? The issue with metadata is that it cannot be conveniently pre-processed within the pipeline itself.

### Demo

#### Setting Up the Portfolio Instruments and Calculating Z-Scores

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import QuantLib as ql
import sklearn
from skfolio import Population, Portfolio, RiskMeasure
from skfolio.datasets import (
    load_bond_dataset,
    load_bond_metadata_dataset,
    load_usd_rates_dataset,
)
from skfolio.moments import LedoitWolf
from skfolio.optimization import (
    MaximumDiversification,
    MeanRisk,
    ObjectiveFunction,
)
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import (
    BlackLitterman,
    EmpiricalPrior,
    NonLinearPrior,
    PortfolioInstruments,
    ReturnsProcessor,
    calculate_sensis,
    price_df,
)
from sklearn.compose import make_column_selector
from tqdm import tqdm

from quantlib_adapter import QLInstrumentAdapter, QLMarketContext, parse_ql_date

sklearn.set_config(enable_metadata_routing=True)

In [ ]:
PCT = 0.01
BPS = PCT * PCT

bond_prices = load_bond_dataset()
bond_metadata = load_bond_metadata_dataset()
rates = (
    load_usd_rates_dataset()
    .reindex(bond_prices.index)
    .interpolate()
    .add_prefix("rate_")
    * PCT
)

X = prices_to_returns(bond_prices)

In [ ]:
def create_fixed_rate_bond(
    issue_date,
    maturity,
    coupon,
) -> ql.FixedRateBond:
    tenor = ql.Period(ql.Semiannual)
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    business_convention = ql.Unadjusted
    date_generation = ql.DateGeneration.Backward
    month_end = False
    settlement_days = 0  # If settlement days is greater than 0, the dirty price of a bond will no longer equal its NPV
    face_value = 100
    day_count = ql.Thirty360(ql.Thirty360.BondBasis)

    schedule = ql.Schedule(
        parse_ql_date(issue_date),
        parse_ql_date(maturity),
        tenor,
        calendar,
        business_convention,
        business_convention,
        date_generation,
        month_end,
    )
    bond = ql.FixedRateBond(
        settlement_days, face_value, schedule, [coupon * PCT], day_count
    )

    return bond

Now we have set up the code to create bonds in quantlib. However, SKfolio does not know anything about QuantLib. That's why we need our separate skfolio_quantlib_adapter package. It contains some relatively basic code that adapts the `Instrument` and `MarketContext` classes to work with QuantLib. All we need to do to make the two compatible here is to pass our QuantLib bonds to the `QLInstrumentAdapter` class constructor before the adding it to our `PortfolioInstruments`.

In [ ]:
# Create a portfolio of instruments using the the parameters defined in the bond_metadata dataset
portfolio_instruments = PortfolioInstruments(
    **{
        isin: QLInstrumentAdapter(
            create_fixed_rate_bond(
                row["issue_date"],
                row["maturity_date"],
                row["coupon_rate"],
            )
        )
        for isin, row in bond_metadata.iterrows()
    }
)

In [6]:
# Calculate the accrued coupons and add them to the clean prices to get dirty prices
accrued_coupons = {}
for date in bond_prices.index:
    ql.Settings.instance().evaluationDate = parse_ql_date(date)
    accrued_coupons[date] = {}
    for isin, bond in portfolio_instruments.items():
        accrued_coupons[date][isin] = bond.accruedAmount()

accrued_coupons = pd.DataFrame.from_dict(accrued_coupons, orient="index")
dirty_bond_prices = bond_prices + accrued_coupons

In [7]:
# Keep track of obersvables used in quantlib pricings
ql_env = {}

# Create discount curve from SOFR OIS rates
index = ql.Sofr()
settlement_days = 2

ois_helpers = ql.RateHelperVector()

cal = ql.BespokeCalendar("my-cal")
cal.addWeekend(ql.Saturday)
cal.addWeekend(ql.Sunday)

for tenor, quote in rates.iloc[0].items():
    q = ql.SimpleQuote(quote)
    ql_env[tenor] = q
    ois_helpers.append(
        ql.OISRateHelper(
            settlement_days,
            ql.Period(tenor),
            ql.QuoteHandle(q),
            index,
            paymentFrequency=ql.Annual,
        )
    )

sofr_curve = ql.PiecewiseLinearZero(
    0,
    cal,
    ois_helpers,
    ql.Actual360(),
)

discount_curve = ql.RelinkableYieldTermStructureHandle(sofr_curve)

times = np.linspace(0.0, discount_curve.maxTime(), 2500)
px.line(x=times, y=[discount_curve.zeroRate(t, ql.Continuous).rate() for t in times])

In [ ]:
# Set up the pricing engines for each bond in the portfolio and link them to a zero spreaded curve.
# We store the curve and z-spread quote in the ql_env dictionary for later use
for isin, bond in portfolio_instruments.items():
    spread_quote = ql.SimpleQuote(0.0)
    risky_curve = ql.RelinkableYieldTermStructureHandle(
        ql.ZeroSpreadedTermStructure(discount_curve, ql.QuoteHandle(spread_quote))
    )
    ql_env[f"curve_{isin}"] = risky_curve
    bond.setPricingEngine(ql.DiscountingBondEngine(risky_curve))
    ql_env[f"z_spread_{isin}"] = spread_quote

In [ ]:
def calculate_z_spread(isin, market_price):
    # The root-finding function for finding spread that makes the bond
    # price close to market price

    spread = ql_env[f"z_spread_{isin}"]
    bond = portfolio_instruments[isin]

    def z_spread_func(z_spread):
        spread.setValue(z_spread)
        return bond.NPV() - market_price

    # Create and configure the Brent solver
    accuracy = 1e-6
    min = -1e-5
    max = 0.1
    solver = ql.Ridder()
    solver.setMaxEvaluations(10000)

    # Solve the spread
    z_spread = solver.solve(z_spread_func, accuracy, spread.value(), min, max)

    return z_spread

In [10]:
# Convert the bond prices to z_spreads

z_spreads = {}

for date, row in tqdm(list(dirty_bond_prices.join(rates, how="inner").iterrows())):
    market_context = QLMarketContext(date, ql_env=ql_env, **row)
    market_context.update_ql_env()
    z_spreads[date] = {}
    for id, market_price in row[bond_prices.columns].items():
        isin = id.replace("price_", "")
        z_spread = calculate_z_spread(isin, market_price)
        z_spreads[date]["z_spread_" + isin] = z_spread

z_spreads = pd.DataFrame.from_dict(z_spreads, orient="index")
px.line(z_spreads)

100%|██████████| 1231/1231 [02:06<00:00,  9.75it/s]


We're going to do a quick sanity check to make sure we've computed the z-spreads correctly. To do so, we reprice our bonds using our z-spreads and see if our new prices match the original prices within a given error. To do so, we'll be using a little convenience function I've added to SKfolio called `price_df`. What it does is it loops over every row in a dataframe, updates the reference_market_context provided with the data in the row, and reprices the portfolio. It then concatenates all the new portfolio prices into a new df and returns it. 

In [ ]:
# Sanity check - reprice bonds using calculated z_spreads
repriced_bonds = price_df(
    z_spreads.join(rates), portfolio_instruments, QLMarketContext(ql_env=ql_env)
)

# Check max repricing error
max_pricing_error = np.max(np.abs(repriced_bonds - dirty_bond_prices))
print("Largest pricing error:", max_pricing_error)

# These bonds are quoted to tenths of a cent, thus the maximum repricing error cannot be more than 0.0005 (half a cent)
assert max_pricing_error < 0.0005

Largest pricing error: 1.2663235764875935e-05


Now that we have our pricers set up and transformed our data into something a little closer to an "invariant," we can start building return distributions for our portfolio. We do this using a new prior I introduced, the `NonLinearPrior`. The `NonLinearPrior` is fitted with a history of market quotes, not a history of returns. It then takes the latest market quotes (the reference quotes) and uses it to price the portfolio. These are the reference prices with respect to which returns will be measured. The same reference quotes used to make are reference prices are then multiplied by the historical returns of the quotes to build a distribution of the following day's market quotes. On this distribution of market_quotes we call the `price_df` function to turn them back into prices and then simply divide them by our reference_prices to create a return distribution. 

In [ ]:
pricing_context = QLMarketContext(ql_env=ql_env).update_from_series(
    z_spreads.join(rates).iloc[-1]
)

prior = NonLinearPrior(
    portfolio_instruments,
    reference_market_context=pricing_context,
)
returns = pd.DataFrame(
    prior.fit(X, market_quotes=z_spreads).return_distribution_.returns,
    columns=bond_prices.columns,
)

px.histogram(returns, labels={"value": "Bond Return"})

As a sanity check to make sure the above actually works, we compare the above price distribution to that created using the first-order approximation of the bond returns. The first-order approximation is simply the change in z-spread multiplied by the given bond's duration.

In [ ]:
z_spread_movements = (
    z_spreads.iloc[-1] * (1 + prices_to_returns(z_spreads))
) - z_spreads.iloc[-1]
durations = calculate_sensis(
    pricing_context, portfolio_instruments, keys=z_spreads.columns
)
estimated_bond_movements = z_spread_movements @ durations
px.histogram(estimated_bond_movements, labels={"value": "Bond Return"})

It looks pretty similar, but as one final check let's plot the difference in the returns estimated by our NonLinearPrior and our first-order approximation against the size of the z-spread move:

In [ ]:
z_spread_movements.columns = estimated_bond_movements.columns
df = (
    pd.concat(
        [
            z_spread_movements.stack().droplevel(0),
            estimated_bond_movements.stack().droplevel(0),
            returns.stack().droplevel(0),
        ],
        keys=["z_spread_movement", "estimated_bond_movement", "repriced_bond_movement"],
        axis=1,
    )
    .reset_index()
    .rename({"index": "isin"}, axis=1)
)

px.scatter(
    x=df["z_spread_movement"],
    y=df["estimated_bond_movement"] - df["repriced_bond_movement"],
    color=df["isin"],
    title="Approximation Error vs Z-Spread Movement",
    labels={"x": "Z-Spread Movement", "y": "Approximation Error"},
)

As expected, the error is close to zero when the z-spread movement is close to 0 and grows quadratically with the size of the z-spread movement. This is just as expected for a first-order approximation and shows that our pricer is accurately capturing the convexity of the bonds.

#### Most Basic Example -- Using Z-Scores as Market Quotes

In [ ]:
pricing_context = QLMarketContext(ql_env=ql_env).update_from_series(
    z_spreads.join(rates).iloc[-1]
)

max_div_model = MaximumDiversification(
    prior_estimator=NonLinearPrior(
        portfolio_instruments,
        reference_market_context=pricing_context,
    ).set_fit_request(market_quotes=True)
)

max_div_model.fit(X, market_quotes=z_spreads)
return_dist = pd.DataFrame(
    max_div_model.prior_estimator_.return_distribution_.returns,
    columns=max_div_model.feature_names_in_,
)
max_div_portfolio = max_div_model.predict(return_dist)
max_div_portfolio.plot_composition()

In [16]:
max_div_portfolio.plot_returns_distribution()

#### Estimating the Moments of the Market Quotes

We would like to be able to use SKfolio's large suite of moment estimators for the prior of the market_quotes. But how can we relate the market_quotes' moments to the moments of the return distribution of our portfolio instruments? We can only rely on the empirical distribution when our moment estimators are the empirical estimators `EmpiricalMu` and `EmpiricalCovariance`. Otherwise, we will have to make the following approximation. Let $X$ be a vector of market quotes, and $f(X): \mathbb{R}^n \rightarrow \mathbb{R}^k$ is a function that maps the market quotes to the prices of the instruments in our portfolio. We are interested in $\mathbb{E}[f(X)]$ and $\text{Cov}[f(X)] = \mathbb{E}\left[(f(X) - \mathbb{E}[f(X)])(f(X) - \mathbb{E}[f(X)])^T\right]$. If we assume our pricing function is approximately linear $f(X) = a_{(k \times n)}X_{(1 \times n)}+b_{(1 \times k)}$ then we have $\mathbb{E}[f(X)] = a \mathbb{E}[X] + b$ and $\text{Cov}[f(X)] = \text{Cov}[aX + b] = \text{Cov}[aX] = a\text{Cov}[X]a^T$. 

If we wished we could even use a second-order approximation for $\mathbb{E}[f(X)]$. If $f(X) \approx a + bX + cXX^T$ then $\mathbb{E}[f(X)] = a + b\mathbb{E}[X] + c\mathbb{E}[XX^T] = a + b\mathbb{E}[X] + c(\text{Cov}[X] + \mathbb{E}[X]\mathbb{E}[X]^T)$. The more moments of $X$ we have available to us, the better our approximation for the moments of $f(X)$. For now, SKfolio's prior API only includes two moments, so that's as far as we can go for now. However, one can still manipulate higher moments of the distribution of $X$ directly using `EntropyPooling`.

To demonstrate this manipulation of the moments of the market_quotes we decrease the expected returns of the z_spread of one of the bonds in the portfolio using the `BlackLittermanPrior`. We also try to improve the conditioning of the covariance matrix by applying LedoitWolf shrinkage to the MarketQuote returns covariance matrix.

In [ ]:
max_sharpe_model = MeanRisk(
    risk_measure=RiskMeasure.STANDARD_DEVIATION,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=NonLinearPrior(
        portfolio_instruments,
        reference_market_context=pricing_context,
        market_quotes_prior=BlackLitterman(
            views=["z_spread_US606822BH67 == -0.02"],
            prior_estimator=EmpiricalPrior(covariance_estimator=LedoitWolf()),
        ),
    ).set_fit_request(market_quotes=True),
)

max_sharpe_model.fit(X, market_quotes=z_spreads)
return_dist = pd.DataFrame(
    max_sharpe_model.prior_estimator_.return_distribution_.returns,
    columns=max_sharpe_model.feature_names_in_,
)
max_sharpe_portfolio = max_sharpe_model.predict(return_dist)
max_sharpe_portfolio.plot_composition()

In [ ]:
print(
    "Condition number of Max Sharpe return covariance:",
    np.linalg.cond(max_sharpe_model.prior_estimator_.return_distribution_.covariance),
    "\nCondition number of Max Div return covariance:",
    np.linalg.cond(max_div_model.prior_estimator_.return_distribution_.covariance),
)

Condition number of Max Sharpe return covariance: 200.01993891690634 
Condition number of Max Div return covariance: 464.05106706678777


As expected, the max-sharpe portfolio allocates most of its portfolio to the bond whose z-spread is expected to shrink significantly. Furthermore, the covariance matrix of the returns of the portfolio instruments is better conditioned in the max-sharpe model than in the max-diversification model since we shrunk the covariance of the market quote returns in the former.

#### Adding Rates to the Market Quotes

So far we've only built our returns distribution using z-spread movements, but what if we want to add rates? Doing so is fairly straightforward -- we only have to join the rates df to the z_spreads df and pass the joined df to the market_quotes metadata parameter when fitting. However, we're going to make it a little trickier by setting rates to be almost 0 accross the curve.

In [ ]:
# Update the discount curve to be almost 0

pricing_context.update(
    {
        rate: 1 * BPS  # Quantlib does not like it when spreads are zero
        for rate in rates.columns
    }
)
pricing_context.update_ql_env()

For reference, we first create a basic returns distribution using only z-spreads. Below we plot the distribution of the returns of the equally weighted portfolio.

In [ ]:
prior = NonLinearPrior(
    portfolio_instruments=portfolio_instruments,
    reference_market_context=pricing_context,
).set_fit_request(market_quotes=True)

prior.fit(X, market_quotes=z_spreads)
return_dist = pd.DataFrame(
    prior.return_distribution_.returns, columns=prior.feature_names_in_
)
Portfolio(
    return_dist, weights=np.ones(len(return_dist.columns)) / len(return_dist.columns)
).plot_returns_distribution()

Now we add rates to our market_quotes. Notice that the distribution does not change. Obviously we're doing something wrong here -- clearly our portfolio should be more volatile when it is also exposed to rates and not just z-spreads. 

What's happening is that because our reference rates are essentially zero, when we build our market_quotes distribution by multiplying the reference rates by the historical returns of the rates, the rates remain pretty much zero. 

In [ ]:
prior.fit(X, market_quotes=z_spreads.join(rates))
return_dist = pd.DataFrame(
    prior.return_distribution_.returns, columns=prior.feature_names_in_
)
Portfolio(
    return_dist, weights=np.ones(len(return_dist.columns)) / len(return_dist.columns)
).plot_returns_distribution()

A better way to handle this scenario is to use the arithmetic returns $\text{IR}_t - \text{IR}_{t-1}$ of the rates instead of the linear returns $\frac{\text{IR}_t}{\text{IR}_{t-1}} - 1$. To do so we pass a `ReturnsProcessor` instance to the NonLinearPrior. When we create the `ReturnsProcessor` instance we define what returns are to be applied to which columns (among other parameters) and then the NonLinearPrior calls its `df_to_returns` method to convert `market_quotes` to returns and then converts the returns back to quotes again using the `returns_to_df` method. 

In [ ]:
select_rates = make_column_selector(pattern="^rate_")
select_z_scores = make_column_selector(pattern="^z_score_")
return_types = {select_rates: "arithmetic", select_z_scores: "linear"}

prior = NonLinearPrior(
    portfolio_instruments=portfolio_instruments,
    returns_processor=ReturnsProcessor(return_types=return_types),
    reference_market_context=pricing_context,
).set_fit_request(market_quotes=True)

prior.fit(X, market_quotes=z_spreads.join(rates))
return_dist = pd.DataFrame(
    prior.return_distribution_.returns, columns=prior.feature_names_in_
)
Portfolio(
    return_dist, weights=np.ones(len(return_dist.columns)) / len(return_dist.columns)
).plot_returns_distribution()

Something worth bearing in mind is that the `ReturnsProcessor` only affects how the `NonLinearPrior` computes the distribution of the market quotes internally and does not affect the type of return used for the ultimate return distribution of the portfolio instruments. The NonLinearPrior only ever uses linear returns for its return distribution. 

#### Stress Test with Vine Copula

In [ ]:
from skfolio.distribution import VineCopula
from skfolio.prior import NonLinearPrior, SyntheticData

vine = VineCopula(central_assets=["z_score_US606822BR40"], n_jobs=-1, random_state=0)


def get_copula_model_returns(**copula_params) -> pd.DataFrame:
    return (
        NonLinearPrior(
            portfolio_instruments=portfolio_instruments,
            reference_market_context=pricing_context,
            market_quotes_prior=SyntheticData(
                n_samples=10_000,
                sample_args=copula_params,
            ),
        )
        .set_fit_request(market_quotes=True)
        .fit(X, market_quotes=z_spreads)
        .return_distribution_.returns
    )


portfolio_weights = np.ones(len(portfolio_instruments)) / len(portfolio_instruments)  # Equally weighted portfolio
Population(
    [
        Portfolio(
            get_copula_model_returns(),
            name="Unstressed Vine Copula Model",
            weights=portfolio_weights,
        ),
        Portfolio(
            get_copula_model_returns(central_parameter="z_spread_US606822BR40", conditioning={"z_spread_US606822BR40": 0.2}),
            name="Stressed Vine Copula Model",
            weights=portfolio_weights,
        ),
    ]
).plot_returns_distribution()

C:\Users\bwink\projects\skfolio\src\skfolio\distribution\multivariate\_vine_copula.py:566: UserWarning:

When performing conditional sampling, it is recommended to set conditioning assets as central during Vine Copula construction. The following conditioning assets were not set as central: {0}. This can be achieved by using the `central_assets` parameter.

